In [5]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import datetime as dt
from datetime import timedelta
from darksky import forecast
key = "e202a3b89ed3182ce3d86552661df096" #darksky key
from sklearn.linear_model import LogisticRegression

In [6]:
train = pd.read_csv("Fire Data - Train.csv")

In [7]:
train['Lat'] = np.zeros(len(train))
train['Long'] = np.zeros(len(train))
train['Month'] = np.zeros(len(train))
train['Temp'] = np.zeros(len(train))
train['WindSpd'] = np.zeros(len(train))
train['WindGust'] = np.zeros(len(train))
train['Humidity'] = np.zeros(len(train))
train['Precipitation'] = np.zeros(len(train))
for i in np.arange(0,len(train)):
    #Convert Lat/Long to 2 separate variables
    latlong = (((train['Lat/Long'].iloc[i]).strip()).strip('][')).split(",")
    train['Lat'].iloc[i] = float(latlong[0])
    train['Long'].iloc[i] = float(latlong[1])
    #Add "Month" variable
    timestamp = train['Date'].iloc[i].split("/")
    train['Month'].iloc[i] = float(timestamp[0])+(int(timestamp[1])/31)
    #Add weather variables
    t = dt(int(timestamp[2]),int(timestamp[0]),int(timestamp[1]),12).isoformat()
    try:
        weather = forecast(key,train['Lat'].iloc[i],train['Long'].iloc[i],time=t)
        train['Temp'].iloc[i] = weather.temperature
        train['WindSpd'].iloc[i] = weather.windSpeed
        train['WindGust'].iloc[i] = weather.windGust
        train['Humidity'].iloc[i] = weather.humidity
        train['Precipitation'].iloc[i] = weather.precipIntensity
    except:
        #weather = forecast(key,round(train['Lat'].iloc[i])+1,round(train['Long'].iloc[i])+1,time=t)
        train['Temp'].iloc[i] = "BAD RESPONSE"

//anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [8]:
train

,Date,Acres Burned,Lat/Long,Lat,Long,Month,Temp,WindSpd,WindGust,Humidity,Precipitation
0,11/16/2018,10,"[37.12857, -122.12036]",37.128570,-122.120360,11.516129,66.74,2.16,6.20,0.20,0.0
1,11/15/2018,20,"[37.81111, -121.7944]",37.811110,-121.794400,11.483871,54.46,3.36,5.48,0.28,0.0
2,11/14/2018,150,"[34.30166667, -119.09916667]",34.301667,-119.099167,11.451613,76.67,10.73,20.47,0.07,0.0
3,11/14/2018,20,"[37.61158, -121.94182]",37.611580,-121.941820,11.451613,60.56,4.24,8.06,0.20,0.0
4,11/13/2018,147,"[34.18075, -117.43184]",34.180750,-117.431840,11.419355,65.08,8.48,19.72,0.08,0.0
5,11/12/2018,10,"[40.888785, -124.001849]",40.888785,-124.001849,11.387097,59.67,3.03,9.78,0.47,0.0
6,11/12/2018,186,"[34.26734825, -118.64381589]",34.267348,-118.643816,11.387097,68.13,11.83,26.37,0.07,0.0
7,11/8/2018,"96,949","[34.235, -118.70128]",34.235000,-118.701280,11.258065,70.53,12.80,26.66,0.16,0.0
8,11/8/2018,4531,"[34.22685, -118.93444]",34.226850,-118.934440,11.258065,75.07,13.02,24.26,0.11,0.0
9,11/8/2018,1500,"[38.21396, -121.9424]",38.213960,-121.942400,11.258065,67.21,12.50,32.41,0.12,0.0


In [9]:
x_train = train[['Date', 'Lat/Long']]

In [10]:
y_train = train[['Acres Burned']]

In [11]:
#model
logisticRegr = LogisticRegression()
logisticRegr.fit(x_train, y_train)

//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


ValueError: could not convert string to float: '[37.7393, -121.85663]'

In [12]:
#predict
predictions = logisticRegr.predict(x_test)

NameError: name 'x_test' is not defined

In [ ]:
# Use score method to get accuracy of model
score = logisticRegr.score(x_test, y_test)
print(score)